In [1]:
from pathlib import Path
import pandas as pd

In [2]:
folder_path = Path('data')
zip_files = list(folder_path.glob('*.zip'))
print(zip_files)
zip_file = zip_files[-1].absolute()
print(zip_file)

[WindowsPath('data/2023-04-26_00-54-lenta.zip'), WindowsPath('data/2023-04-27_14-27-lenta.zip'), WindowsPath('data/2023-04-30_13-43-lenta.zip')]
c:\works\work_002\data\2023-04-30_13-43-lenta.zip


In [3]:
df = pd.read_csv(zip_file)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30624 entries, 0 to 30623
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Категория      30624 non-null  object 
 1   Название       30624 non-null  object 
 2   Страна         30523 non-null  object 
 3   Скидка         15747 non-null  object 
 4   Цена           30624 non-null  float64
 5   Цена по карте  30622 non-null  float64
dtypes: float64(2), object(4)
memory usage: 1.4+ MB


In [33]:
mask = df['Название'].str.contains('Килька')
kilka = df[mask].copy()
mask2 = kilka['Категория'] == 'Бакалея'
mask3 = kilka['Название'].str.lower()#.contains('лента')
mask3 = mask3.str.contains('лента')
test = kilka[mask2 & mask3].sort_values(by='Цена').copy()
test.reset_index()

,index,Категория,Название,Страна,Скидка,Цена,Цена по карте
0,6893,Бакалея,"Килька ЛЕНТА в томатном соусе, 240г","Россия, 240 г",-31%,94.79,64.99


In [104]:
mask = df['Название'].str.lower().str.contains(r'(?=молоко)(?=.* пастериз)(?=.*без змж)', regex=True)
kilka = df[mask].copy()
mask2 = kilka['Категория'] == 'Бакалея'
mask3 = kilka['Название'].str.lower().str.contains('лента')
# test = kilka[mask2].sort_values(by='Цена').copy()
test = kilka.copy()
test['volume'] = test['Страна'].str.extract('(\d+)').astype(int)
test['Пр. цена'] = round((test['Цена по карте'] / (test['volume'] / 1000)), 2)
mask4 = test['volume']>600
test_min_mean = test[mask4]['Пр. цена'].nsmallest(5).mean()
test_median = test[mask4]['Пр. цена'].median()
test_max_mean = test[mask4]['Пр. цена'].nlargest(5).mean()
print(f'минимум \t{test_min_mean:.2f}')
print(f'медиана \t{test_median:.2f}')
print(f'максимум \t{test_max_mean:.2f}')
print(f'количество позиций: {test.shape[0]}')
display(test.nsmallest(5, columns='Пр. цена'))

минимум 	62.55
медиана 	85.71
максимум 	135.98
количество позиций: 35


,Категория,Название,Страна,Скидка,Цена,Цена по карте,volume,Пр. цена
2517,"Молоко, сыр, яйцо","Молоко пастеризованное 365 ДНЕЙ 2,5%, без змж,...","Россия, 800 мл",NaN,38.19,36.19,800,45.24
1620,"Молоко, сыр, яйцо","Молоко пастеризованное ЛЕБЕДЯНЬМОЛОКО 2,5%, бе...","Россия, 900 г",-19%,70.49,56.99,900,63.32
1603,"Молоко, сыр, яйцо","Молоко пастеризованное ЛЕБЕДЯНЬМОЛОКО 3,2%, бе...","Россия, 900 г",-20%,74.69,59.29,900,65.88
1658,"Молоко, сыр, яйцо","Молоко пастеризованное СЧАСТЛИВОЕ ДЕТСТВО 1,5%...","Россия, 900 мл",NaN,64.79,61.49,900,68.32
1851,"Молоко, сыр, яйцо","Молоко пастеризованное ЭКОНИВА 2,5%, без змж, ...","Россия, 1000 мл",-36%,110.59,69.99,1000,69.99


In [95]:
display(test[test['volume']>600].nlargest(5, columns='Пр. цена'))

,Категория,Название,Страна,Скидка,Цена,Цена по карте,volume,Пр. цена
2446,"Молоко, сыр, яйцо",Молоко пастеризованное PARMALAT Comfort безлак...,"Россия, 900 мл",NaN,164.29,155.99,900,173.32
2454,"Молоко, сыр, яйцо",Молоко ультрапастеризованное ДОМИК В ДЕРЕВНЕ 6...,"Россия, 950 г",NaN,157.89,149.99,950,157.88
2225,"Молоко, сыр, яйцо",Молоко пастеризованное PARMALAT Comfort безлак...,"Россия, 900 мл",NaN,147.39,139.99,900,155.54
3056,"Молоко, сыр, яйцо",Молоко ультрапастеризованное ЭКОНИВА Органичес...,"Россия, 1000 мл",NaN,157.89,149.99,1000,149.99
1803,"Молоко, сыр, яйцо",Молоко ультрапастеризованное ПЕТМОЛ Для капучи...,"Россия, 950 мл",NaN,148.39,140.89,950,148.31


In [137]:
import requests
from bs4 import BeautifulSoup as BS
import lxml
from pprint import pprint

url = 'https://ru.wikipedia.org/wiki/%D0%9F%D0%BE%D1%82%D1%80%D0%B5%D0%B1%D0%B8%D1%82%D0%B5%D0%BB%D1%8C%D1%81%D0%BA%D0%B0%D1%8F_%D0%BA%D0%BE%D1%80%D0%B7%D0%B8%D0%BD%D0%B0'

response = requests.get(url)
soup = BS(response.content, 'lxml')

table = soup.find('table', class_='wikitable')
rows = list(table.find_all('tr'))[2:]

sum_list = []
for row in rows[:]:
    out_list = []
    for index, element in enumerate(row.find_all('td')):
        if index > 1:
            per_year = float(element.get_text(strip=True).replace(',', '.'))
            per_month = per_year / 12
            out_list.append(round(per_month, 2))
        else:
            out_list.append(element.get_text(strip=True))
    sum_list.append(out_list)

table_df = pd.DataFrame(sum_list, columns=['name', 'unit', 'works', 'pensioners', 'children'])
table_df

,name,unit,works,pensioners,children
0,Хлебные продукты (хлебимакаронные изделияв пер...,кг,10.54,8.18,6.47
1,Картофель,кг,8.37,6.67,7.34
2,Овощиибахчевые,кг,9.55,8.17,9.38
3,Фруктысвежие,кг,5.00,3.75,9.84
4,Сахарикондитерские изделияв пересчете на сахар,кг,1.98,1.77,1.82
5,Мясопродукты,кг,4.88,4.50,3.67
6,Рыбопродукты,кг,1.54,1.33,1.55
7,Молокои молокопродукты в пересчете на молоко,кг,24.17,21.48,30.06
8,Яйца,штука,17.50,16.67,16.75
9,"Масло растительное,маргарини другиежиры",кг,0.92,0.83,0.42
